In [ ]:
import sys
import cmath
import math
import os
import h5py
import matplotlib.pyplot as plt   # plots
from matplotlib import cm
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import scipy.sparse as sp
from scipy.optimize import curve_fit

import time
import warnings
import glob

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

import util.libutil as comn
from libra_py import units, data_conv
import libra_py.models.GLVC as GLVC

from libra_py import dynamics_plotting

import libra_py.dynamics.tsh.compute as tsh_dynamics
import libra_py.dynamics.tsh.plot as tsh_dynamics_plot

import libra_py.dynamics.exact.compute as dvr
import libra_py.dynamics.exact.save as dvr_save

import libra_py.packages.cp2k.methods as CP2K_methods

from numpy.fft import fft, ifft
from libra_py import trpes

#from matplotlib.mlab import griddata
#%matplotlib inline 
#warnings.filterwarnings('ignore')

colors = {}
colors.update({"11": "#CB3310", "12": "#E0603B", "13": "#F08566"}) # red
colors.update({"21": "#0077BB", "22": "#4093D2", "23": "#80B0E9"}) # blue
colors.update({"31": "#009988", "32": "#33B8A8", "33": "#66D7C8"}) # teal
colors.update({"41": "#EE7733", "42": "#F59566", "43": "#FDC399"}) # orange
colors.update({"51": "#00FFFF", "52": "#66FFFF", "53": "#99FFFF"}) # cyan
colors.update({"61": "#EE3377", "62": "#F46695", "63": "#FB99B3"}) # magenta
colors.update({"71": "#AA3377", "72": "#BF6695", "73": "#D399B3"}) # purple
colors.update({"81": "#BBBBBB", "82": "#DDDDDD", "83": "#EEEEEE"}) # grey

clrs_index = [
    "11", "21", "31", "41", "51", "61", "71", "81", 
    "12", "22", "32", "42", "52", "62", "72", "82", 
    "13", "23", "33", "43", "53", "63", "73", "83"
]



In [ ]:
plt.rc('axes', titlesize=40)      # fontsize of the axes title
plt.rc('axes', labelsize=40)      # fontsize of the x and y labels
plt.rc('legend', fontsize=25)     # legend fontsize
plt.rc('xtick', labelsize=40)     # fontsize of the tick labels
plt.rc('ytick', labelsize=40)     # fontsize of the tick labels

#plt.rc('figure.subplot', left=0.2)
#plt.rc('figure.subplot', right=0.95)
#plt.rc('figure.subplot', bottom=0.2)
plt.rc('figure.subplot', top=0.88)


font = {'family': 'serif',
        'color':  'blue',
        'weight': 'bold',
        'size': 36,
        }

In [ ]:
def compute_model(q, params, full_id):

    model = params["model"]
    res = None

    if model==1:
        res = GLVC.GLVC(q, params, full_id)
    else:
        pass

    return res

In [ ]:
def potential(q, params):
    full_id = Py2Cpp_int([0,0]) 
    
    return compute_model(q, params, full_id)

In [ ]:
# Spin-boson model
s = units.inv_cm2Ha
freqs = np.array([129.743484,263.643268,406.405874,564.000564,744.784527,
                  961.545250,1235.657107,1606.622430,2157.558683,3100.000000])*s
freqs = freqs.tolist()
g_vals = [0.000563,0.001143,0.001762,0.002446,0.003230,0.004170,0.005358,0.006966,0.009356,0.013444]
eps = 0.02
v0 = 0.001
m = 1836.0 # 1 amu
temperature = 300.0 # K

model_params1 = {"model":1, "model0":1, "nstates":2, "num_osc":10, "coupling_scaling":[1.0, -1.0], 
                 "omega":[freqs]*2, "coupl":[g_vals]*2, "mass":[m]*10, "Ham":[[eps,-v0],[-v0,-eps]],
                 "beta": 1/(units.kB*temperature)}

all_model_params = [model_params1]

#################################
# Give the model used an index
model_indx = 0
################################

model_params = all_model_params[model_indx]

list_states = [x for x in range(model_params["nstates"])]
NSTATES = model_params["nstates"]

In [ ]:
def get_Eadi(_model_indx, q):
    nst = all_model_params[_model_indx]["nstates"]
    res = potential(q, all_model_params[_model_indx])
    H = np.zeros((nst, nst), dtype=complex)
    for i in range(nst):
        for j in range(nst):
            H[i,j] = res.ham_dia.get(i,j)
    E, U = np.linalg.eig(H) 
    idx= np.argsort(E) 
    E = E[idx] 
    U = U[:,idx]

    return H, E

-----

# SBH

### Figure 3

In [ ]:
with h5py.File("SBH/02-NBRA_TSH/fssh_nbra_0_icond_0_batch_0/mem_data.hdf") as f:
    time = np.array(f["time/data"])

nbatches = 5
POP_FSSH = np.zeros((time.shape[0] , all_model_params[model_indx]["nstates"]))
for ibatch in range(nbatches):
    with h5py.File(F"SB/02-NBRA_TSH/fssh_nbra_0_icond_0_batch_{ibatch}/mem_data.hdf") as f:
        POP_FSSH += np.array(f["sh_pop_adi/data"])
POP_FSSH /= nbatches

In [ ]:
with h5py.File("SBH/02-NBRA_TSH/msdm_nbra_0_icond_0_batch_0/mem_data.hdf") as f:
    time = np.array(f["time/data"])

nbatches = 5
POP_mSDM = np.zeros((time.shape[0] , all_model_params[model_indx]["nstates"]))
for ibatch in range(nbatches):
    with h5py.File(F"SBH/02-NBRA_TSH/msdm_nbra_0_icond_0_batch_{ibatch}/mem_data.hdf") as f:
        POP_mSDM += np.array(f["sh_pop_adi/data"])
POP_mSDM /= nbatches

In [ ]:
POP_RPI = []

for i, ipatch in enumerate([1000, 6250, 12500, 20000]):
    data = []
    with open(F"SBH/04-RPI_SUM/POP_NPATCH_{ipatch}_all.dat", 'r') as f:
        for line in f:
            if line.strip():  # skip empty lines
                data.append([float(x) for x in line.strip().split()])
    data = np.array(data)

    # Extract time and populations
    time = data[:, 0]
    pops = data[:, 1:]
    
    POP_RPI.append(pops)

In [ ]:
%matplotlib inline

fig, axs = plt.subplots(1,2, figsize=(24,8))

# Population
axs[0].plot(time*units.au2fs/1000, POP_FSSH[:, 1], color=colors[clrs_index[0]], label="FSSH", lw=5)
axs[0].plot(time*units.au2fs/1000, POP_mSDM[:, 1], color="k", label="mSDM", lw=5)

axs[0].margins(x=0)
axs[0].set_xlabel("Time (ps)")
axs[0].set_ylabel("Population")
axs[0].legend(frameon=False, loc="center right", fontsize=30, bbox_to_anchor=(1.05,0.5))

axs[0].text(5, 0.38, r"$\tau_{e}=$23 ps "+ "\n" + r"$\tau_{g}=$12 ps", color=colors[clrs_index[0]], fontsize=35)
axs[0].text(10.0, 0.82, r"$\tau_{e}=$0.90 ns " + "\n" + r"$\tau_{g}=$5.9 ps", color="k", fontsize=35)

axs[0].margins(x=0)
axs[0].set_xlabel("Time (ps)")
axs[0].set_ylabel("Population")
axs[0].legend(frameon=False, fontsize=35, loc="center right", bbox_to_anchor=(1.05, 0.5), labelspacing=0.25, handlelength=1.0)

## mSDM & RPI
list_rpi_label = [0.05, 0.3, 0.6, 1.0]

axs[1].plot(time*units.au2fs/1000, POP_mSDM[:, 1], label="mSDM", lw=5, color="k")

# RPI
for i, ipatch in enumerate([1000, 6250, 12500, 20000]):
    axs[1].plot(time*units.au2fs/1000, POP_RPI[i][:,1], label=rF"$\gamma=${list_rpi_label[i]} $\mathrm{{fs}}^{{-1}}$", 
                lw=5, color=colors[clrs_index[i]])

axs[1].margins(x=0)
axs[1].set_xlabel("Time (ps)")
axs[1].set_ylabel("Population")
axs[1].legend(frameon=False, fontsize=35, loc="lower left", bbox_to_anchor=(-0.03, -0.07), 
              labelspacing=0.25, handlelength=1.0)

fig.subplots_adjust(wspace=0.3, hspace=0.6)

plt.savefig("PLOTS/SB_pop.png", dpi=500, bbox_inches='tight')

----

# C60

In [ ]:
with h5py.File("C60/02-NBRA_TSH/fssh_nbra_1_icond_0_batch_0/mem_data.hdf") as f:
    time_fssh = np.array(f["time/data"])
    
nbatches = 5
POP_FSSH = np.zeros((time_fssh.shape[0], 55))
for ibatch in range(nbatches):
    with h5py.File(F"C60/02-NBRA_TSH/fssh_nbra_1_icond_0_batch_{ibatch}/mem_data.hdf") as f:
        POP_FSSH += np.array(f["sh_pop_adi/data"][:,:])
        
POP_FSSH /= nbatches

In [ ]:
with h5py.File("C60/02-NBRA_TSH/msdm_nbra_1_icond_0_batch_0/mem_data.hdf") as f:
    time_msdm = np.array(f["time/data"])
    
nbatches = 5
POP_mSDM = np.zeros((time_msdm.shape[0], 55))
for ibatch in range(nbatches):
    with h5py.File(F"C60/02-NBRA_TSH/msdm_nbra_1_icond_0_batch_{ibatch}/mem_data.hdf") as f:
        POP_mSDM += np.array(f["sh_pop_adi/data"][:,:])
        
POP_mSDM /= nbatches

In [ ]:
POP_RPI = []

for i, ipatch in enumerate([20, 200, 500, 1000]):
    data = []
    with open(F"C60/04-RPI_SUM/POP_NPATCH_{ipatch}_all.dat", 'r') as f:
        for line in f:
            if line.strip():  # skip empty lines
                data.append([float(x) for x in line.strip().split()])
    data = np.array(data)

    # Extract time and populations
    time = data[:, 0]
    pops = data[:, 1:]
    
    POP_RPI.append(pops)

In [ ]:
def func(t, a, tau1, tau2):
    return a*np.exp(-t/tau1) + (1-a)*np.exp(-(t/tau2)**2) + max_energy_values[0] - 1.

In [ ]:
## TRPES
def get_TRPES(name_method):
    # Read energies from the sampling first
    istep = 1800
    fstep = 2999
    nst = 55
    E = []
    for step in range(istep, fstep):
        energy_mat = sp.load_npz(F"C60/01-sampling_dynamics/step3/res-mb-sd-DFT/Hvib_ci_{step}_re.npz")
        E.append( np.array( np.diag( energy_mat.todense() ) ) )
    E = np.multiply(E, units.au2ev)    
    
    #print(E[0])
    
    # Compute the TRPES
    Nt = 1001
    emin, emax = 0, 5.0 #eV
    de, sigma_e = 0.01, 0.1
    
    n_e_grid_pts = int((emax - emin)/de)+1
    
    t_grid = np.linspace(0, Nt, Nt)
    e_grid = np.linspace(emin, emax, n_e_grid_pts)
    
    with h5py.File(F"C60/02-NBRA_TSH/fssh_nbra_1_icond_0_batch_0/mem_data.hdf") as f:
        time = np.array(f["time/data"])*units.au2fs
    
    iconds = [1800, 1850, 1900, 1950, 1998]
    
    TRPES = np.zeros((Nt, n_e_grid_pts))
    TRPES_traj = np.zeros((Nt, n_e_grid_pts))
    
    cnt = 0    
    for ibatch, icond in enumerate(iconds):
        with h5py.File(F"C60/02-NBRA_TSH/{name_method}_nbra_1_icond_0_batch_{ibatch}/mem_data.hdf") as f:
            sh_pop = np.array(f["sh_pop_adi/data"][:,:])
            
        for i in range(Nt):
            for j, e_j in enumerate(e_grid):
                for ist in range(nst):
                    e = E[i + icond - istep, ist]
                    smear = np.exp(-(e_j - e) ** 2) / (2 * sigma_e ** 2)
                    TRPES_traj[i, j] += sh_pop[i, ist] * de * smear
        
        TRPES += TRPES_traj
        cnt += 1
    
    TRPES /= cnt
    TRPES /= np.max(TRPES)
    
    return t_grid, e_grid, TRPES

In [ ]:
t_grid, e_grid, TRPES_fssh = get_TRPES('fssh')

In [ ]:
t_grid, e_grid, TRPES_msdm = get_TRPES('msdm')

In [ ]:
## TRPES
def get_TRPES_rpi(ipatch):
    # Read energies from the sampling first

    istep = 1800
    fstep = 2999
    nst = 55
    
    E = []
    for step in range(istep, fstep):
        energy_mat = sp.load_npz(F"C60/01-sampling_dynamics/step3/res-mb-sd-DFT/Hvib_ci_{step}_re.npz")
        E.append( np.array( np.diag( energy_mat.todense() ) ) )
    E = np.multiply(E, units.au2ev)    
    
    # Compute the TRPES
    Nt = 1000
    emin, emax = 0, 5.0 #eV
    de, sigma_e = 0.01, 0.1
    
    n_e_grid_pts = int((emax - emin)/de)+1
    
    t_grid = np.linspace(0, Nt, Nt)
    e_grid = np.linspace(emin, emax, n_e_grid_pts)
    
    with h5py.File(F"C60/02-NBRA_TSH/fssh_nbra_1_icond_0_batch_0/mem_data.hdf") as f:
        time = np.array(f["time/data"])*units.au2fs
    
    iconds = [1800, 1850, 1900, 1950, 1998]
    
    TRPES = np.zeros((Nt, n_e_grid_pts))
    TRPES_traj = np.zeros((Nt, n_e_grid_pts))
    
    cnt = 0
    for ibatch, icond in enumerate(iconds):
        sh_pop = np.loadtxt(F"C60/03-RPI_PATCH/POP_NPATCH_{ipatch}_ibatch_{ibatch}.dat")[:,1:]
        for i in range(Nt):
            for j, e_j in enumerate(e_grid):
                for ist in range(nst):
                    e = E[i + icond - istep, ist]
                    smear = np.exp(-(e_j - e) ** 2) / (2 * sigma_e ** 2)
                    TRPES_traj[i, j] += sh_pop[i, ist] * de * smear
        
        TRPES += TRPES_traj
        cnt += 1
    
    TRPES /= cnt
    TRPES /= np.max(TRPES)
    
    return t_grid, e_grid, TRPES

In [ ]:
t_grid, e_grid, TRPES_rpi1 = get_TRPES_rpi(500)

In [ ]:
t_grid, e_grid, TRPES_rpi2 = get_TRPES_rpi(200)

### Figure 5

In [ ]:
%matplotlib inline

params = {"path_to_energy_files": "C60/01-sampling_dynamics/step3/res-mb-sd-DFT", "dt": 1.0, 
          "prefix": "Hvib_sd_", "suffix": "_re", "istep": 1800, "fstep": 2999}

fig, axs = plt.subplots(2,2, figsize=(20,15.39))

params.update({"prefix": F"Hvib_ci_"})
md_time, energies = CP2K_methods.extract_energies_sparse(params)
energies = energies * units.au2ev

for i in range(1): # Ground
    axs[0][0].plot(md_time, energies[:,i]-energies[:,0], lw=5, color="k")

for i, bnd in enumerate([x for x in range(1,16)]): # X1
    axs[0][0].plot(md_time, energies[:,bnd]-energies[:,0], lw=2, color="gray")

for i, bnd in enumerate([x for x in range(16,55)]): # X2
    axs[0][0].plot(md_time, energies[:,bnd]-energies[:,0], lw=2, color="mediumpurple")
 
axs[0][0].plot([], [], lw=5, color="mediumpurple", label=r"$X_{2}$")
axs[0][0].plot([], [], lw=5, color="gray", label=r"$X_{1}$")
axs[0][0].plot([], [], lw=5, color="k", label=r"Ground")

axs[0][0].legend(loc="lower right", fontsize=35, frameon=False, labelspacing=0.4, handlelength=1.0, bbox_to_anchor=(1.05, 0.02))

axs[0][0].set_ylabel('Energy (eV)')
axs[0][0].set_xlabel('Time (ps)')

axs[0][0].set_yticks([0, 2.0, 4.0], ["0.0", "2.0", "4.0"])
axs[0][0].margins(x=0)
axs[0][0].set_xticks([0,500, 1000], [0,0.5, 1.0])

nac_files = glob.glob('C60/01-sampling_dynamics/step3/res-mb-sd-DFT/Hvib_ci*im*')
target_numbers = set(str(i) for i in range(1800, 2999))
nac_files_sub = [file for file in nac_files if any(num in file for num in target_numbers)]

for c2, nac_file in enumerate(nac_files):
    nac_mat = sp.load_npz(nac_file).todense().real
    if c2==0:
        nac_ave = np.zeros(nac_mat.shape)
    nac_ave += np.abs(nac_mat)
nac_ave *= 1000*units.au2ev/c2
nstates = nac_ave.shape[0]
axs[0][1].imshow(np.flipud(nac_ave), cmap='hot', extent=(0,nstates,0,nstates), vmin=0, vmax=20)
axs[0][1].set_xlabel('State index')
axs[0][1].set_ylabel('State index')
axs[0][1].set_yticks([0, 50])
colorbar = fig.colorbar(axs[0][1].imshow(np.flipud(nac_ave), cmap='hot', extent=(0,nstates,0,nstates)), ax=axs[0][1], shrink=1.0)
colorbar.ax.tick_params(labelrotation=90)
colorbar.set_ticks([0, 100])

#axs[0][1].set_title("NAC (meV) ")

ax2 = axs[0][1].twinx()
ax2.set_ylabel("NAC (meV) ", labelpad=100)
ax2.set_yticks([])  # Remove tick marks
ax2.set_yticklabels([])  # Remove tick labels

# Population, X2
axs[1][0].plot(time_msdm*units.au2fs, np.sum(POP_mSDM[:,16:], axis=1), label=F"mSDM", lw=5, color="k")

for i, ipatch in enumerate([20, 200, 500, 1000]):
    axs[1][0].plot(time, np.sum(POP_RPI[i][:,16:], axis=1), lw=5, color=colors[clrs_index[i]], label=rF"$\gamma=${ipatch/1000}")
    
axs[1][0].margins(x=0)
axs[1][0].yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.1f}'))
axs[1][0].set_yticks([0.9, 1.0])
axs[1][0].set_xlabel("Time (fs)")
axs[1][0].set_ylabel(r"Population $X_{2}$")
axs[1][0].text(155, 0.91, r"$\tau=7.1$ ps", fontsize=35)
axs[1][0].set_xticks([0,500, 1000], [0,0.5, 1.0])

# X1
axs[1][1].plot(time_msdm*units.au2fs, np.sum(POP_mSDM[:,1:16], axis=1), label=F"mSDM", lw=5, color="k")

for i, ipatch in enumerate([20, 200, 500, 1000]):
    axs[1][1].plot(time, np.sum(POP_RPI[i][:,1:16], axis=1), lw=5, 
                   color=colors[clrs_index[i]], label=rF"$\gamma=${ipatch/1000} $\mathrm{{fs}}^{{-1}}$")
    
axs[1][1].margins(x=0)
axs[1][1].yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:.1f}'))
axs[1][1].set_yticks([0.0, 0.1])
axs[1][1].set_xlabel("Time (fs)")
axs[1][1].set_ylabel(r"Population $X_{1}$")
axs[1][1].legend(frameon=False, fontsize=35, loc="upper left", bbox_to_anchor=(-0.04,1.06),
                 ncol=1, labelspacing=0.2, columnspacing=0.4, handlelength=1.0)
axs[1][1].set_xticks([0,500, 1000], [0,0.5, 1.0])

fig.tight_layout()
#fig.subplots_adjust(wspace=0.3, hspace=0.6)

plt.savefig("PLOTS/c60_sta_pop.png", dpi=500, bbox_inches='tight')

-----

### Figure 5

In [ ]:
%matplotlib inline

params = {"path_to_energy_files": "C60/01-sampling_dynamics/step3/res-mb-sd-DFT", "dt": 1.0, 
          "prefix": "Hvib_sd_", "suffix": "_re", "istep": 1800, "fstep": 2999}

fig, axs = plt.subplots(1,3, figsize=(25,7))

# E vs. t, FSSH
cmap = cm.get_cmap('rainbow')
img = axs[0].imshow(TRPES_fssh.T, extent=[t_grid.min(), t_grid.max(), e_grid.min(), e_grid.max()],
                origin='lower', aspect='auto', cmap=cmap, vmin=0, vmax=1)

axs[0].set_xlim(t_grid.min(), t_grid.max())
axs[0].set_ylim(e_grid.min(), e_grid.max())
axs[0].set_xticks(np.arange(t_grid.min(), t_grid.max() + 1, 500))
axs[0].tick_params(axis='x', labelsize=40)
axs[0].tick_params(axis='y', labelsize=40)

max_energy_indices = np.argmax(TRPES_fssh, axis=1) 
max_energy_values = e_grid[max_energy_indices]
popt, _ = curve_fit(func, t_grid, max_energy_values)

axs[0].plot(t_grid, func(t_grid, popt[0], popt[1], popt[2]), color="k", lw=5, ls="--")

axs[0].text(60, 3.7, r"$\tau_{g}=$ 21 fs", fontsize=40)
axs[0].text(170, 1.8, r"$\tau_{e}=$ 0.4 ps", fontsize=40)

axs[0].set_title("NBRA-FSSH", fontsize=40)
axs[0].set_ylabel(F'Energy (eV)', fontsize=40)
axs[0].set_xlabel(F'Time (ps)', fontsize=40)
axs[0].set_xticks([0, 500, 1000], ["0", "0.5", "1.0"])
axs[0].set_yticks([2,4])

# E vs. t, mSDM
img = axs[1].imshow(TRPES_msdm.T, extent=[t_grid.min(), t_grid.max(), e_grid.min(), e_grid.max()],
                origin='lower', aspect='auto', cmap=cmap, vmin=0, vmax=1)

axs[1].set_xlim(t_grid.min(), t_grid.max())
axs[1].set_ylim(e_grid.min(), e_grid.max())
axs[1].set_xticks(np.arange(t_grid.min(), t_grid.max() + 1, 500))
axs[1].tick_params(axis='x', labelsize=40)
axs[1].tick_params(axis='y', labelsize=40)

max_energy_indices = np.argmax(TRPES_msdm, axis=1) 
max_energy_values = e_grid[max_energy_indices]
popt, _ = curve_fit(func, t_grid, max_energy_values)

axs[1].plot(t_grid, func(t_grid, popt[0], popt[1], popt[2]), color="k", lw=5, ls="--")

axs[1].text(60, 3.7, r"$\tau_{g}=$ 36 fs", fontsize=40)
axs[1].text(250, 1.8, r"$\tau_{e}=$ 3.3 ps", fontsize=40)

axs[1].set_title("NBRA-mSDM", fontsize=40)
#axs[1].set_ylabel(F'Energy (eV)', fontsize=40)
axs[1].set_xlabel(F'Time (ps)', fontsize=40)
axs[1].set_xticks([0, 500, 1000], ["0", "0.5", "1.0"])
axs[1].set_yticks([2,4])

# E vs. t, 0.5 fs^-1
img = axs[2].imshow(TRPES_rpi1.T, extent=[t_grid1.min(), t_grid1.max(), e_grid.min(), e_grid.max()],
                    origin='lower', aspect='auto', cmap=cmap, vmin=0, vmax=1)

# Create a divider for the existing axes
divider = make_axes_locatable(axs[2])
cax = divider.append_axes("right", size="5%", pad=0.1)  # Adjust size and padding as needed

# Place colorbar in the new axis
cbar = fig.colorbar(img, cax=cax)
cbar.set_label("Intensity", fontsize=40, labelpad=5)
cbar.ax.yaxis.set_label_coords(1.2, 0.5)
cbar.ax.tick_params(labelrotation=90)
cbar.set_ticks([0, 1])

axs[2].set_xlim(t_grid1.min(), t_grid1.max())
axs[2].set_ylim(e_grid.min(), e_grid.max())
axs[2].set_xticks(np.arange(t_grid1.min(), t_grid1.max() + 1, 500))
axs[2].tick_params(axis='x', labelsize=40)
axs[2].tick_params(axis='y', labelsize=40)

max_energy_indices = np.argmax(TRPES_rpi1, axis=1) 
max_energy_values = e_grid[max_energy_indices]
popt, _ = curve_fit(func, t_grid1, max_energy_values)

axs[2].plot(t_grid1, func(t_grid1, popt[0], popt[1], popt[2]), color="k", lw=5, ls="--")

axs[2].text(60, 3.7, r"$\tau_{g}=$ 49 fs", fontsize=40)
axs[2].text(250, 1.8, r"$\tau_{e}=$ 7.1 ps", fontsize=40)

axs[2].set_title(r"RPI, $\gamma=0.5\: \mathrm{fs}^{-1}$", fontsize=40)
#axs[2].set_ylabel(F'Energy (eV)', fontsize=40)
axs[2].set_xlabel(F'Time (ps)', fontsize=40)
axs[2].set_xticks([0, 500, 1000], ["0", "0.5", "1.0"])
axs[2].set_yticks([2,4])

#fig.tight_layout()
fig.subplots_adjust(wspace=0.15)

plt.savefig("PLOTS/c60_trpes.png", dpi=500, bbox_inches='tight')

### Figure S1

In [ ]:
%matplotlib inline

ndof = 10
q = MATRIX(ndof,1)

list_active_dof = [0, 4, 9]
list_xlim = [[-0.6, 0.6], [-0.6, 0.6], [-0.6, 0.6]]

list_omega = [129.743484,263.643268,406.405874,564.000564,744.784527,961.545250,1235.657107,1606.622430,2157.558683,3100.000000]

fig, axs = plt.subplots(1,3, figsize=(15,5))

for i, mode in enumerate([0,4,9]):
    for idof in range(ndof):
        q.set(idof, 0, 0.0)
    
    dx = (list_xlim[i][1] - list_xlim[i][0])/200.
    qs = [list_xlim[i][0] + dx*ix for ix in range(200)]
    
    Hs = []
    Es = []
    for q_temp in qs:
        q.set(list_active_dof[i], 0, q_temp)
        H, E = get_Eadi(0, q)
        Hs.append(np.float64(H))
        Es.append(np.float64(E))
    Hs = np.array(Hs)
    Es = np.array(Es)

    axs[i].plot(qs, Hs[:,0,0]*units.au2ev, lw=5, color=colors[clrs_index[0]], ls="dotted", zorder=10)
    axs[i].plot(qs, Hs[:,1,1]*units.au2ev, lw=5, color=colors[clrs_index[1]], ls="dotted", zorder=10)
    
    axs[i].plot(qs, Es[:,0]*units.au2ev, lw=7, color=colors[clrs_index[2]])
    axs[i].plot(qs, Es[:,1]*units.au2ev, lw=7, color=colors[clrs_index[3]])
        
    axs[i].margins(x=0)
    axs[i].set_title(rF"$\omega_{{{mode}}}={list_omega[mode]:.0f} \:\mathrm{{cm}}^{{-1}}$")
    axs[i].set_xlabel(r"$q$ (Bohr)")
    axs[i].yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.1f}'))

#axs[0].set_ylabel("Energy (Ha)")
axs[0].set_ylabel("Energy (eV)")

axs[0].plot([], [], lw=5, label=r"$H_{00}$", color=colors[clrs_index[0]], ls="dotted", zorder=10)
axs[0].plot([], [], lw=5, label=r"$H_{11}$", color=colors[clrs_index[1]], ls="dotted", zorder=10)    
axs[0].plot([], [], lw=5, label=r"$E_{0}$", color=colors[clrs_index[2]])
axs[0].plot([], [], lw=5, label=r"$E_{1}$", color=colors[clrs_index[3]])
axs[0].legend(frameon=False, loc='center')

fig.tight_layout()

plt.savefig("PLOTS/pes_sb.png", dpi=500, bbox_inches='tight')